In [1]:
import pandas as pd

Import all the files

In [2]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [3]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [23]:
visits_cart = pd.merge(visits, cart, how ='left')
print(visits_cart.head())

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time  
0                 NaT  
1 2017-01-26 14:44:00  
2 2017-08-20 08:31:00  
3                 NaT  
4                 NaT  


Step 3: How long is `visits_cart`?

In [8]:
print(len(visits_cart))

2000


Step 4: How many timestamps are null for `cart_time`?

In [11]:
time_null = len(visits_cart[visits_cart.cart_time.isnull()])
print(time_null)

1652


Step 5: What percentage only visited?

In [21]:
only_visit = len(visits_cart[visits_cart.cart_time.isnull()])
pct_only_visit = only_visit/len(visits_cart)

print(pct_only_visit)

0.826


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [35]:
incl_checkout = pd.merge(visits_cart, checkout, how='left')
# print(incl_checkout)

count_cart = len(incl_checkout[~incl_checkout.cart_time.isnull()])
count_check = len(incl_checkout[~incl_checkout.checkout_time.isnull()])

pct_only_cart = 1- count_check/count_cart
print(pct_only_cart)

0.3505747126436781


Step 7: Merge it all together

In [33]:
full = pd.merge(incl_checkout, purchase, how = 'left')


Step 8: % of users who got to checkout but did not purchase

In [37]:
count_check = len(full[~full.checkout_time.isnull()]) # people who were checking out
count_purch = len(full[~full.purchase_time.isnull()]) # people who purchased
pct_only_check = 1-count_purch/count_check
print(count_check, count_purch, pct_only_check) 

334 252 0.2455089820359282


Step 9: check each part of the funnel, let's print all 3 of them again

In [39]:
print(pct_only_visit)
print(pct_only_cart)
print(pct_only_check)

0.826
0.3505747126436781
0.2455089820359282


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: Using the giant merged DataFrame all_data that you created, let’s calculate the average time from initial visit to final purchase. Add a column that is the difference between purchase_time and visit_time.

In [46]:
full['time_on'] = full.purchase_time - full.visit_time

Step 11: examine the results

In [47]:
print(full.time_on.head(20))

0                NaT
1    0 days 00:44:00
2                NaT
3                NaT
4                NaT
5                NaT
6                NaT
7                NaT
8                NaT
9                NaT
10               NaT
11               NaT
12               NaT
13               NaT
14   0 days 00:38:00
15               NaT
16               NaT
17               NaT
18               NaT
19               NaT
Name: time_on, dtype: timedelta64[ns]


Step 12: average time to purchase

In [48]:
average = full.time_on.mean()
print(average)

0 days 00:43:12.380952380
